In [2]:
import tensorflow as tf
import json
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np
import random

<h3>Preprocessing data</h3>

In [3]:
def tokenize(sentence):
   return nltk.word_tokenize(sentence)

stemmer = PorterStemmer()
def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    
    return bag

<h3>Create Dataset</h3>

In [4]:
with open('intents.json', 'r') as f:
    intents = json.load(f)
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'items',
   'patterns': ['Which items do you have?',
    'What kinds of items are there?',
    'What do you sell?'],
   'responses': ['We sell coffee and tea', 'We have coffee and tea']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Can I pay with Paypal?',
    'Are you cash only?'],
   'responses': ['We 

In [5]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']

all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))
tags

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']

In [6]:
X_train, y_train = [], []
for (pattern, tag) in xy:
    bag = bag_of_words(pattern, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

<h3>Train model</h3>

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(len(X_train[0]),)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(tags), activation='softmax'),
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=32, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 6s 6s/step - loss: 2.0205 - accuracy: 0.0769
Epoch 2/1000
1/1 [==============================] - 0s 20ms/step - loss: 1.9319 - accuracy: 0.2692
Epoch 3/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9456 - accuracy: 0.1154
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9318 - accuracy: 0.1538
Epoch 5/1000
1/1 [==============================] - 0s 21ms/step - loss: 1.9196 - accuracy: 0.1923
Epoch 6/1000
1/1 [==============================] - 0s 21ms/step - loss: 1.9750 - accuracy: 0.1154
Epoch 7/1000
1/1 [==============================] - 0s 30ms/step - loss: 1.9279 - accuracy: 0.1923
Epoch 8/1000
1/1 [==============================] - 0s 31ms/step - loss: 1.8482 - accuracy: 0.1154
Epoch 9/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.9464 - accuracy: 0.1923
Epoch 10/1000
1/1 [==============================] - 0s 26ms/step - loss: 1.8476 - accuracy: 0.2308
Epoch 11/10

<h3>Testing Model</h3>

In [1]:
bot_name = 'Sam'
print("Let's chat! type 'quit' to exit")
while True:
    sentence = input("You: ")
    if sentence == 'quit':
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])

    pred = model.predict(X)
    prob = pred.argmax(axis=1)
    tag = tags[prob[0]]

    if pred[0][prob[0]] > 0.90:
      for intent in intents['intents']:
          if tag == intent['tag']:
              print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
      print(f"{bot_name}: I don't understand...")

Let's chat! type 'quit' to exit
